* Name: Abdulrahman Mohamed Kamr
* Phone Number: 01148706623
* mail: abdokamr94@gmail.com


## This task divided into Three parts
#### Part 1: On this Jupyter notebook named:`01_Code.ipynb`, and this include all processing from greb data to printing the recommended job function.

#### Part 2: On sypder named :`02_Code.py`, and this part  is a script as a RESTful API service include all processes to print the recommended job function on web by Flask.
#### Part 3: The Report named as `03_Report`

# Part 1:

## Contents:
* 1- Import Necessary Libraries
* 2- Read In and Explore the Data
* 3- Cleaning Data
* 4- Make a Similarity function
* 5- Make a Recommendation job functions


### 1- Import Necessary Libraries


In [220]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import pairwise_distances

import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer as ps
from nltk import WordNetLemmatizer as wn

from nltk.tokenize import sent_tokenize , word_tokenize
import string
import re

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import pairwise_distances


%matplotlib inline

## 2- Reading In and Explore the Data

In [221]:
data = pd.read_csv('jobs_data.csv', index_col = 0)
data = data.drop(['industry'], axis = 1) # removing useless features 
# data = data.drop(['Unnamed: 0', 'industry'], axis = 1) # removing useless features 
data.head()

,title,jobFunction
0,Full Stack PHP Developer,"['Engineering - Telecom/Technology', 'IT/Softw..."
1,CISCO Collaboration Specialist Engineer,"['Installation/Maintenance/Repair', 'IT/Softwa..."
2,Senior Back End-PHP Developer,"['Engineering - Telecom/Technology', 'IT/Softw..."
3,UX Designer,"['Creative/Design/Art', 'IT/Software Developme..."
4,Java Technical Lead,"['Engineering - Telecom/Technology', 'IT/Softw..."


In [222]:
# what is the shape of the dataset?
print('the data has {} rows and has {} cols'.format(len(data) , len(data.columns)))


the data has 10870 rows and has 2 cols


In [223]:
data.describe()

,title,jobFunction
count,10870,10870
unique,3230,836
top,Graphic Designer,"['IT/Software Development', 'Engineering - Tel..."
freq,107,1489


* The data has a duplicate title and jobFunction and the unique of `title = 3230` and unique of `jobFunction = 836` from 10870

* the most frequent in title is `Graphic Designer = 107` 
* the most frequent in jobFunction is `[IT/Software Development, Engineering - Telecom/Technology, Customer Service/Support] = 1489` 


In [224]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10870 entries, 0 to 10869
Data columns (total 2 columns):
title          10870 non-null object
jobFunction    10870 non-null object
dtypes: object(2)
memory usage: 254.8+ KB


In [225]:
data.sample(5) # see a sample of the dataset to get an idea of the variables

,title,jobFunction
3928,UI/UX Web Designer,"['Engineering - Telecom/Technology', 'Creative..."
10323,Maintenance Engineer,"['Installation/Maintenance/Repair', 'Engineeri..."
3761,Senior Digital Marketing Executive,"['Media/Journalism/Publishing', 'Marketing/PR/..."
3890,Infant Class Teacher,['Education/Teaching']
5771,CRM Developer,"['Engineering - Telecom/Technology', 'IT/Softw..."


In [226]:
# show the title feature
data['title']

0                                 Full Stack PHP Developer
1                  CISCO Collaboration Specialist Engineer
2                            Senior Back End-PHP Developer
3                                              UX Designer
4                                      Java Technical Lead
5                               Technical Support Engineer
6                                     Senior iOS Developer
7                                      Mechanical Engineer
8           Real Estate Sales Specialist - 10th of Ramadan
9                                         School Principal
10                             Senior Sales Representative
11                                              Accountant
12                                  Indoor Sales Executive
13                          PHP Full-Stack - Joomla Expert
14                               English Teacher Assistant
15                                   Marketing Coordinator
16                               Senior Business Develop

In [227]:
# there's Arabic words in the data set
print( data['title'][9804])
print( data['title'][10859])

Arabic KG Teacher - مُدرسة لغة عربية رياض أطفال
Arabic KG Teacher - مُدرسة لغة عربية رياض أطفال


In [228]:
# number of unique title feature
uni_title = data['title'].unique()
len(uni_title)

3230

In [229]:
# number of unique jobfunction feature
uni_jobfunction = data['jobFunction'].unique()
len(uni_jobfunction)

836

* So, we have a duplicate data .. but we remove the duplicate `title` rows.

## 3- Cleaning Data
* 1- Removing Duplicate
* 2- Removing puncituation
* 3- Removing stopwords
* 4- Tokenization
* 5- Lower case

In [230]:
unduplicata_data = data.drop_duplicates(subset='title') # delete duplicated by title row
unduplicata_data.shape

(3230, 2)

In [231]:
unduplicata_data.head()

,title,jobFunction
0,Full Stack PHP Developer,"['Engineering - Telecom/Technology', 'IT/Softw..."
1,CISCO Collaboration Specialist Engineer,"['Installation/Maintenance/Repair', 'IT/Softwa..."
2,Senior Back End-PHP Developer,"['Engineering - Telecom/Technology', 'IT/Softw..."
3,UX Designer,"['Creative/Design/Art', 'IT/Software Developme..."
4,Java Technical Lead,"['Engineering - Telecom/Technology', 'IT/Softw..."


In [232]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [233]:
stopwords = nltk.corpus.stopwords.words('english')
stopwords[:5]

['i', 'me', 'my', 'myself', 'we']

In [234]:
# function for make some of NLP functions

# this func for title 
def clean_text_without_splitting(text):
    text = re.sub("[^A-Za-z]", " ", text.strip())                                   # Remove non english words
    text = "".join([word for word in text if word not in string.punctuation])       # Removing puncituation
    tokenize = re.split('\W+' , text)                                               # Tokenization    
    text = [word.lower() for word in tokenize if word not in stopwords]             # Removing stopwords , lower case
    text = [word for word in text if len(word)]                                     # Remove the empty string
    text = " ".join(word for word in text)                                          # Make it as a sentence     
    return text

# this func for jobFunction 
def clean_text_with_splitting(text):
    text = re.sub("[^A-Za-z]", " ", text.strip())                                   # Remove non english words
    text = "".join([word for word in text if word not in string.punctuation])       # Removing puncituation
    tokenize = re.split('\W+' , text)                                               # Tokenization    
    text = [word.lower() for word in tokenize if word not in stopwords]             # Removing stopwords , lower case
    text = [word for word in text if len(word)]                                     # Remove the empty string    
    return text


In [235]:
data['clean_title'] = data['title'].apply(lambda x : clean_text_without_splitting(x)) # without splitting mean as string
data['clean_jobFunction'] = data['jobFunction'].apply(lambda x : clean_text_with_splitting(x)) # with splitting 

# delete the old cols
data = data.drop(['title', 'jobFunction'], axis=1)
data.head(10)

,clean_title,clean_jobFunction
0,full stack php developer,"[engineering, telecom, technology, it, softwar..."
1,cisco collaboration specialist engineer,"[installation, maintenance, repair, it, softwa..."
2,senior back end php developer,"[engineering, telecom, technology, it, softwar..."
3,ux designer,"[creative, design, art, it, software, developm..."
4,java technical lead,"[engineering, telecom, technology, it, softwar..."
5,technical support engineer,"[it, software, development, engineering, telec..."
6,senior ios developer,"[engineering, telecom, technology, it, softwar..."
7,mechanical engineer,"[engineering, mechanical, electrical]"
8,real estate sales specialist th ramadan,"[sales, retail]"
9,school principal,"[education, teaching, administration, operatio..."


## 4- Make a Similarity functions

In [236]:
# this func return the similarity between q1 and q2
def jaccard_similarity_1(q1, q2):
    intersection = set(q1).intersection(set(q2))
    union = set(q1).union(set(q2))
    return len(intersection)/len(union)
    
# take each titel in clean title (q2) and make a similarity between q2 and q1(the input) and call jaccard_similarity_1
def jaccard_similarity_2(q1):
    title_list = data['clean_title'] # second qurery for make jac_sim => q2    
    sim = []
    for i in range(len(title_list)):
        q2 = title_list[i]
        sim.append((jaccard_similarity_1(q1, q2), i))
    return sim


# take a list of tuples [(sim_num, index)] and sorted them descending by sim_num
# make a list of all indeces from high similarity to low 
# make a list of top titles and return them
def sorting_rev(list_of_sim_and_index):
    sorted_sim_list = sorted(list_of_sim_and_index , reverse= True)
    
    # make a list of high similarity of spicific title
    index_of_high_sim = []
    title_of_high_sim = []
    for i in range(len(sorted_sim_list)):
        index_of_high_sim.append(sorted_sim_list[i][1]) # sorted_sim_list => (sim, index)
    
    # here we iterate on list of indeces and take just 6 job function     
    flag = 0
    for i in range(len(index_of_high_sim)):
        if(flag):
            break
        for j in range(len(data['clean_jobFunction'][index_of_high_sim[i]])):
            if (data['clean_jobFunction'][index_of_high_sim[i]][j] not in title_of_high_sim and (len(data['clean_jobFunction'][index_of_high_sim[i]][j]) > 1)):
                title_of_high_sim.append(data['clean_jobFunction'][index_of_high_sim[i]][j])
                if(len(title_of_high_sim)> 6):
                    flag =1 
                    break
                
    
    sample_of_title_of_high_sim = title_of_high_sim[:6]
    return sample_of_title_of_high_sim

# for printing the top 6 titles
def print_it_as_atitle(list_of_job_title):
    for st in list_of_job_title:
        print(st.title())
    
    

## 5- Make a Recommendation job functions


In [237]:
# call all function by this function to return the top 6 titles
def recommendation_job_functions(query):
    clean_query = clean_text_without_splitting(query)        # text preprocessing for input
    list_of_indx = jaccard_similarity_2(clean_query)         # return list of tuple for (sim, index)
    list_of_job_functions = sorting_rev(list_of_indx)        # return list of sorted indices
    final_answer = print_it_as_atitle(list_of_job_functions) # print the final answer as a Title string
    return final_answer

#### Simple Examples

In [238]:
q = "Machine Learninig Engineer"
print(recommendation_job_functions(q))

Engineering
Mechanical
Electrical
Other
Installation
Maintenance
None


In [239]:
q1 = "java developer"
print(recommendation_job_functions(q1))

Engineering
Telecom
Technology
It
Software
Development
None


In [240]:
q2 = "Graphic Designer"
print(recommendation_job_functions(q2))

Creative
Design
Art
Marketing
Pr
Advertising
None


# Enter your job title

In [241]:
# print('The input title is :')
query = str(input())
print('The input title is :', query)
print('-'*50)
# IOS Developer
print('The related top 6 of Job Functions are:')

print(recommendation_job_functions(query))


IOS Developer
The input title is : IOS Developer
--------------------------------------------------
The related top 6 of Job Functions are:
It
Software
Development
Engineering
Telecom
Technology
None
